In [ ]:
import numpy as np
from pcdiff import knn_graph, estimate_basis, build_grad_div, laplacian, coords_projected, gaussian_weights, weighted_least_squares, batch_dot

def grad_curvature(pos, k_fit, k_compare, kernel_width=1, regularizer=1e-8, shape_regularizer=None):

    compare_neighbours = k_compare
    fit_neighbours = knn_graph(pos, k_fit)

    normal, x_basis, y_basis = estimate_basis(pos, fit_neighbours)

    row, col = fit_neighbours
    k = (row == 0).sum()

    coords = coords_projected(pos, normal, x_basis, y_basis, fit_neighbours, k)

    dist = LA.norm(pos[col] - pos[row], axis=1)
    weights = gaussian_weights(dist, k, kernel_width)

    if shape_regularizer is None:
        wls = weighted_least_squares(coords, weights, k, regularizer)
    else:
        wls, wls_shape = weighted_least_squares(coords, weights, k, regularizer, shape_regularizer)

    normal = np.tile(normal[:, None], (1, k, 1)).reshape(-1, 3)
    local_pos = pos[col] - pos[row]
    C = wls * batch_dot(local_pos, normal)
    # Format as sparse matrix

    # The gradient of f at (0, 0) will be
    # df/dx|(0, 0) = [1, 0, c1 + 2*c3*0 + c4*0] = [1, 0, c1]
    # df/dy|(0, 0) = [0, 1, c2 + c4*0 + 2*c5*0] = [0, 1, c2]
    # Hence, we can use the row in wls that outputs c1 and c2 for the gradient
    # in x direction and y direction, respectively
    grad_row = np.stack([row * 2, row * 2 + 1], axis=1).flatten()
    grad_col = np.stack([col]*2, axis=1).flatten()
    grad_values = np.stack([wls[:, 1], wls[:, 2]], axis=1).flatten()

    grad_x = C[:,1] + 2 * C[:,3] * coords[:,0] + C[:,4] * coords[:,1]
    grad_y = C[:,2] + 2 * C[:,5] * coords[:,1] + C[:,4] * coords[:,0]
    grad_xx = 2 * C[:,3]
    grad_xy = C[:,4]
    grad_yy = 2 * C[:,5]
    
    

    # Create gradient matrix
    grad = coo_matrix((grad_values, (grad_row, grad_col)), shape=(pos.shape[0] * 2, pos.shape[0]))

    # Divergence
    if shape_regularizer is not None:
        wls = wls_shape
    vector_mapping = fit_vector_mapping(pos, normal, x_basis, y_basis, (row, col), wls, coords)

    # Store as sparse tensor 
    grad_vec = grad_values.reshape(-1, 1, 2)
    div_vec = (grad_vec @ vector_mapping).flatten()
    div_row = np.stack([row] * 2, axis=1).flatten()
    div_col = np.stack([col * 2, col * 2 + 1], axis=1).flatten()
    div = coo_matrix((div_vec, (div_row, div_col)), shape=(pos.shape[0], pos.shape[0] * 2))

    return grad, div